In [ ]:
import copy
import glob
import numpy as np
import os
import PIL

In [ ]:
from skimage.measure import block_reduce

In [ ]:
import verdict

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
matplotlib.style.use( '/Users/zhafen/repos/clean-bold/clean-bold.mplstyle' )

# Parameters

In [ ]:
example_texture = 'redstone_block'

In [ ]:
source_dir = '/Users/zhafen/data/minecraft/versions/1.19.2/assets/minecraft/textures/block'
output_dir = '/Users/zhafen/data/minecraft/textures/block'

# Load Textures

In [ ]:
texture_fps = glob.glob( os.path.join( source_dir, '*' ) )
textures = {}
for texture_fp in texture_fps:
    block_id = os.path.splitext( os.path.basename( texture_fp ) )[0]
    
    try:
        textures[block_id] = plt.imread( texture_fp )
    except PIL.UnidentifiedImageError:
        continue

In [ ]:
fig = plt.figure()
ax = plt.gca()

ax.imshow( textures[example_texture] )
ax.set_axis_off()

# Extract Representative Colors

In [ ]:
texture_colors = verdict.Dict()
for block_id, texture in textures.items():
    texture_color = block_reduce( texture, block_size=(texture.shape[0], texture.shape[1], 1 ), func=np.mean )
    texture_colors[block_id] = texture_color[0][0]

In [ ]:
# Air doesn't have a texture, but we'll add it as fully translucent white for completeness
# Cave air will be black and opaque
texture_colors['air'] = np.array([ 1., 1., 1., 0., ])
texture_colors['cave_air'] = np.array([ 0., 0., 0., 1., ])

In [ ]:
# Lava flow is also known as lava
texture_colors['lava'] = np.array([ 1., 1., 1., 1., ])
texture_colors['lava'][:3] = texture_colors['lava_flow']

In [ ]:
# Water coloor input manually
texture_colors['water'] = np.array([ 14, 80, 255, 255. ]) / 255.

In [ ]:
for block_id, block_color in copy.deepcopy( texture_colors.items() ):
    
    block_id_split = block_id.split( '_' )
    
    # Fence colors will be plank colors
    if block_id_split[-1] == 'planks':
        wood_type = '_'.join( block_id_split[:-1] )
        
        texture_colors['{}_fence'.format( wood_type )] = block_color
       
    # Torches will be colored as air
    if 'torch' in block_id:
        texture_colors[block_id] = texture_colors['air']
        texture_colors['wall_{}'.format( block_id )] = texture_colors['air']

In [ ]:
color_fp = os.path.join( output_dir, 'colors.h5' )
texture_colors.to_hdf5( color_fp )

In [ ]:
fig = plt.figure()
ax = plt.gca()

ax.imshow( [[ texture_colors[example_texture], ],] )
ax.set_axis_off()